<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

'2.11.0'

In [2]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')

553467096/553467096 [==============================] - 23s 0us/step


In [3]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [40]:
for i in range(len(vgg16.layers)):
  vgg16.layers[i].trainable=False
  print(vgg16.layers[i].name)

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
input_x = tf.keras.Input(shape=(224, 224, 3))
x_1 = vgg16.get_layer('input_1')(input_x)

x_2 = vgg16.get_layer('block1_conv1')(x_1)
x_3 = vgg16.get_layer('block1_conv2')(x_2)
x_4 = vgg16.get_layer('block1_pool')(x_3)


x_5 = vgg16.get_layer('block2_conv1')(x_4)
x_6 = vgg16.get_layer('block2_conv2')(x_5)
x_7 = vgg16.get_layer('block2_pool')(x_6)

x_8 = vgg16.get_layer('block3_conv1')(x_7)
x_9 = vgg16.get_layer('block3_conv2')(x_8)
x_10 = vgg16.get_layer('block3_conv3')(x_9)
x_11 = vgg16.get_layer('block3_pool')(x_10)

x_12 = vgg16.get_layer('block4_conv1')(x_11)
x_13 = vgg16.get_layer('block4_conv2')(x_12)
x_14 = vgg16.get_layer('block4_conv3')(x_13)
x_15 = vgg16.get_layer('block4_pool')(x_14)

x_16 = vgg16.get_layer('block5_conv1')(x_15)
x_17 = vgg16.get_layer('block5_conv2')(x_16)
x_18 = vgg16.get_layer('block5_conv3')(x_17)
x_19 = vgg16.get_layer('block5_pool')(x_18)

x_20 = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1')(x_19)
x_21 = tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool')(x_20)

x_22 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1')(x_21)
x_23 = tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool')(x_22)


x_24 = tf.keras.layers.Flatten()(x_23)
x_25 = tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer')(x_24)
x_26 = tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')(x_25)


In [ ]:
growth_rate =  32
filters =  32
n_dense_blocks = 4
n_dense_block_layers = 12

def dense_block(x, n_layers, filters, drop_rate=0.2, kernel_1=1, kernel_2=3):
    
  previous_input = x
  for i in range(n_layers):
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Conv2D(filters=4*filters, kernel_size=kernel_1, activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=drop_rate)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_2, activation='relu', padding='same')(x)
    x = tf.keras.layers.Dropout(rate=drop_rate)(x)
    x = tf.keras.layers.Concatenate(axis=-1)([previous_input, x])  #Concatenate channel wise
    previous_input = x

  return x


def transition_layer(x, kernel=1, drop_rate=0.2):
  x = tf.keras.layers.BatchNormalization(axis=-1)(x)
  x = tf.keras.layers.Conv2D(filters=x.shape[-1]//2, kernel_size=1, activation='relu')(x)
  x = tf.keras.layers.Dropout(rate=drop_rate)(x)
  x = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2)(x)
  # x = tf.keras.layers.GlobalAveragePooling2D(pool_size=2, strides=2)
  # x = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
  return x


input_x = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.layers.Conv2D(filters=64, activation='relu', kernel_size=7, strides=2, padding='same', input_shape=(224, 224, 3))(input_x)
x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
# x = tf.keras.layers.GlobalMaxPool2D(pool_size=3, strides=2)(x)

for i in range(n_dense_blocks): 
  print(i, x.shape)
  x = dense_block(x, n_dense_block_layers, filters)
  x = transition_layer(x)

print(i, x.shape)


x = dense_block(x, 32, filters)
print(i, x.shape)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
output = tf.keras.layers.Dense(units=len(train_ds.class_names), activation='softmax')(x)

model = tf.keras.models.Model(input_x, output)

In [31]:
vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

model_1 = tf.keras.Sequential([
    vgg16_short, 
    tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool'),
    tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer'),
    tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')
])
model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_8 (Functional)        (None, 7, 7, 512)         14714688  
                                                                 
 block6_conv1 (Conv2D)       (None, 7, 7, 1024)        4719616   
                                                                 
 block6_pool (MaxPooling2D)  (None, 3, 3, 1024)        0         
                                                                 
 block7_conv1 (Conv2D)       (None, 2, 2, 512)         4719104   
                                                                 
 block7_pool (MaxPooling2D)  (None, 1, 1, 512)         0         
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_layer (Dense)         (None, 4096)             

In [33]:
model_2 = tf.keras.models.Model(inputs=vgg16.input, outputs=[vgg16_short.get_layer('block4_conv2').output,
                                                            vgg16_short.get_layer('block4_conv3').output,
                                                            vgg16_short.get_layer('block5_conv3').output,
                                                            model_1.get_layer('block6_conv1').output,
                                                            model_1.get_layer('block7_conv1').output])

ValueError: ignored